In [1]:

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

- REMINDER: It will take 4-5 minutes to run the whole process for a large dataset.

In [2]:
# you might need to add a few of your own
!pip install requests
import requests
import pprint as pp
import datetime
from datetime import date 
from datetime import datetime
import math
# you might need to add a few of your own
import time #for sleepness
import pandas as pd

### retrivel data

In [3]:

api_key = '6f7693fa-bb8c-4c9c-836b-528fb0bc4d26' #Replace with your key

#make api request
def make_api_request(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.json().get('error')}")
        return None
    
#get company detailed information (postcode) for geo-analysis for Question1 while just company number info for Question2.
def process_company_data(companies, data, detailed=False):
    if detailed:
        for company in data:
            address = company.get('registered_office_address', {})
            companies.append({
                'company_name': company.get('company_name'),
                'company_number': company.get('company_number'),
                'date_of_creation': company.get('date_of_creation'),
                'locality': address.get('locality'),
                'postal_code': address.get('postal_code')
            })
    else:
        for company in data:
            companies.append(company.get('company_number'))

            
# use a function to achieve industry filteraion           
def call_api_with_pagination(api_key, sic_codes, max_companies=400, detailed=False):
    base_url = "https://api.companieshouse.gov.uk/advanced-search/companies"
    headers = {"Authorization": api_key}
    page_size = 20  #set the max number of items in each page. only when page_size=20 it works othersie just return 20 results. have no idea why.
    start_index = 0
    all_companies = []

    #q1 and q2 share the same industry filteration process, but different further analysis.
    while True:
        params = { 
            "company_status": "active",
            "sic_codes": sic_codes,
            "incorporated_from": "2014-01-01",
            'company_type': 'ltd',
            "items_per_page": page_size,
            "start_index": start_index
        }

        data = make_api_request(base_url, headers, params)
        if data:
            companies = data.get('items', [])
            process_company_data(all_companies, companies, detailed)

            if len(all_companies) >= max_companies or len(companies) < page_size:
                break

            start_index += page_size
            
            # to respect API rate limits
            time.sleep(1)  
        else:
            break  

    if len(all_companies) < max_companies:
        print(f"Reached only {len(all_companies)} companies, less than the maximum of {max_companies}.")
   
    return all_companies


#for qustion1, i want to analyze geo-distribution of tech companies from Business and domestic software development,
#Information technology consultancy activities and Data processing, hosting and related activities, these three sectors.
sic_codes_1 = "62012, 62020, 63110"

#set the maximum number of 4000 to get a extensice analysis. and get detailed geo-info for each company.
companies = call_api_with_pagination(api_key, sic_codes_1, 4000, True)
df = pd.DataFrame(companies)
print(df)



                         company_name company_number date_of_creation  \
0                   PROJECT TRACK LTD       09229012       2014-09-22   
1                INTUITECTURE LIMITED       09230259       2014-09-23   
2                            ZUSO LTD       09229931       2014-09-22   
3          INTEGRITY PARTNERS LIMITED       09230028       2014-09-23   
4     SADASHIVAM TECHNOLOGIES LIMITED       09230819       2014-09-23   
...                               ...            ...              ...   
3995             THIRTY 1 DEGREES LTD       12138835       2019-08-05   
3996                      HAYNAKU LTD       12139575       2019-08-05   
3997              HIMITSU LAB LIMITED       12140200       2019-08-06   
3998                ARACARI SALES LTD       12139503       2019-08-05   
3999          EL ELYON CONSULTING LTD       12140362       2019-08-06   

                locality postal_code  
0              Orpington     BR5 2LQ  
1              Liverpool     L18 1DG  
2     

In [4]:
#have a look at the dataset.
df.head()

,company_name,company_number,date_of_creation,locality,postal_code
0,PROJECT TRACK LTD,09229012,2014-09-22,Orpington,BR5 2LQ
1,INTUITECTURE LIMITED,09230259,2014-09-23,Liverpool,L18 1DG
2,ZUSO LTD,09229931,2014-09-22,Leeds,LS25 1NB
3,INTEGRITY PARTNERS LIMITED,09230028,2014-09-23,Bishop's Stortford,CM23 3PA
4,SADASHIVAM TECHNOLOGIES LIMITED,09230819,2014-09-23,Hounslow,TW3 1NJ


In [5]:
#i want granularity to be the district(outward post code), 
#extract characters before space.
df['outward_code'] = df['postal_code'].str.split(' ', expand=True)[0]


most_common_locality = df['locality'].value_counts()
print("Most common locality:", most_common_locality)
most_common_outward_code = df['outward_code'].value_counts()
print("Most common outward code:", most_common_outward_code)


Most common locality: London           1090
Manchester         59
Leeds              54
Hove               52
Birmingham         51
                 ... 
Rushden             1
Berwick             1
Lambeth             1
Bude                1
North Shields       1
Name: locality, Length: 754, dtype: int64
Most common outward code: WC2H    122
EC1V     91
N1       85
BN3      56
W1W      45
       ... 
CF23      1
PE3       1
DE74      1
TN23      1
TF1       1
Name: outward_code, Length: 1296, dtype: int64


### visualization part

In [7]:

!pip install geopandas
!pip install folium
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import pandas as pd

# i get uk postcode(+longtitude, latitude) data from this website:https://www.opendoorlogistics.com/downloads/. 
# unfortunately, only England map.
gdf = gpd.read_file('/home/jovyan/CMSE114332023-4SS1SEM1/2023-week11-final-assignment-PG/Districts.json')



# count the number of company in each district
company_counts = df['outward_code'].value_counts().reset_index()
company_counts.columns = ['outward_code', 'count']

# merge my company dataset and the map data
gdf = gdf.merge(company_counts, left_on='name', right_on='outward_code', how='left')

# create folium map
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10)  # set the initial coordinates -- London

# create heatmap layer
heat_data = [[row['geometry'].centroid.y, row['geometry'].centroid.x, row['count']] 
             for index, row in gdf.iterrows() if 'count' in row and row['count'] > 0]

HeatMap(heat_data, radius=4, blur=0.1, max_zoom=4).add_to(m)

HeatMap(heat_data).add_to(m)


m.save('england_heatmap.html')
m

